In [1]:
#from nipype.interfaces import niftyreg
import numpy as np
from skimage import io
from pathlib import Path
import re
import ants
from skimage.transform import resize
from tqdm import tqdm
from skimage.morphology import skeletonize_3d, binary_dilation, binary_closing
from scipy.ndimage import distance_transform_edt
import tifffile as tif
from scipy.ndimage import binary_fill_holes 
import cc3d
from scipy.io import loadmat, savemat
import skan
import sknw
import networkx as nx
import pickle
import os
import matplotlib.pyplot as plt

# Define connected componnet removal

In [2]:
def remove_small_comps_3d(image, thresh = 500):
    """
    

    Parameters
    ----------
    image : binary np array with uint8 elements
        3d numpy matrix, connected components will be removed form this image
    thresh : int64
        smallest connected components to keep

    Returns
    -------
    np.array with uint8 elements, binary
        binary image with connected components below the threshold removed.

    """
    img_lab, N = cc3d.connected_components(image,return_N=True)
    unique, counts = np.unique(img_lab, return_counts=True)
    unique_keep = unique[counts>thresh]
    unique_keep = np.delete(unique_keep,[0])
    img_filt = np.zeros(img_lab.shape).astype('int8')
    img_filt[np.isin(img_lab,unique_keep)] = 1
    return img_filt.astype('uint8')   

def fill_holes(img,thresh=100):
    #res = np.zeros(img.shape)
    for i in np.unique(img)[::-1]:
        _tmp = (img==i)*1.0
        _tmp = _tmp.astype('int8')
        _tmp = remove_small_comps_3d(_tmp,thresh=thresh)
        img[_tmp==1] = i
    res = img.astype('int8')
    return res

# Get mean predictions

In [3]:
directory = Path('matt_preds')
files  = directory.glob('*_mean.npy')
files = sorted([x.as_posix() for x in files])

In [4]:
len(files)

390

# High bias low varience segmentation

With removal of connected components under 500 pixels

In [5]:
min_prob = 0.75
max_var = 0.1
for file in tqdm(files[::-1]):
    if not os.path.exists(re.sub('mean','seg',file)):
        mean = np.load(file)
        std = np.load(re.sub('mean','std',file))
        seg = np.zeros(mean.shape[1:])
        seg[(mean[1,:,:,:] > min_prob) * (std[1,:,:,:] < max_var)] = 1
        seg[(mean[2,:,:,:] > min_prob) * (std[2,:,:,:] < max_var)] = 2
        seg = seg.astype('int8')
        seg = (seg==1)*1
        seg = fill_holes(seg)
        np.save(re.sub('mean','seg',file),seg)
        #savemat(re.sub('mean.npy','seg.mat',file),{'FinalImage':fill_holes(binary_dilation(binary_dilation(seg)))})
        #tif.imwrite(re.sub('mean.npy','seg.tif',file),seg)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 390/390 [00:00<00:00, 73940.18it/s]


# ANTs registration

## Upsample raw images

In [6]:
image_path = Path('../TBI')
images = list(image_path.glob('*_*/XYZres*?[0-9].tif'))
images = sorted([x.as_posix() for x in images])
len(images)

432

In [7]:
for i in tqdm(range(len(images))[::-1]):
    if not os.path.exists('matt_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name)):
        image = io.imread(images[i])
        image = np.swapaxes(image,0,1)
        image = np.swapaxes(image,1,3)
        image = resize(image,(2,507,507,252),preserve_range=True)
        zeros = np.zeros((1,507,507,252)).astype('uint16')
        image = np.append(image,zeros,axis = 0)
        image = image.astype('float16')
        io.imsave('matt_preds/'+re.sub('.tif','_upsamp.tif',Path(images[i]).name),image)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 432/432 [00:00<00:00, 40956.11it/s]


In [8]:
images[i]

'../TBI/20200217_11/XYZres100.tif'

## reshape numpy seg files and resave as tif

In [9]:
directory_seg = Path('matt_preds')
files_seg  = list(directory_seg.glob('*_seg.npy'))
files_seg = sorted([x.as_posix() for x in files_seg])

In [10]:
for i in tqdm(range(len(files_seg))):
    if not os.path.exists(re.sub('.npy','.tif',files_seg[0])):
        tmp = np.load(files_seg[i])
        tmp = np.reshape(tmp,(1,507,507,252))
        io.imsave(re.sub('.npy','.tif',files_seg[0]),tmp)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 390/390 [00:00<00:00, 103891.94it/s]


## Register images and transform masks

In [16]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001_upsamp.tif'))
images = sorted([x.as_posix() for x in images])#[0:2]
images[0]

'matt_preds/XYZ1_0001_upsamp.tif'

In [17]:
for i in tqdm(range(len(images))):
    if os.path.exists(re.sub('upsamp','seg',images[i])):
        if not os.path.exists(re.sub('seg','seg_warped',re.sub('upsamp','seg',images[i]))):
            fix = ants.image_read(re.sub('_0001','',images[i]))
            mov = ants.image_read(images[i])
            fix_mask = ants.image_read(re.sub('upsamp','seg',re.sub('_0001','',images[i])))
            mov_mask = ants.image_read(re.sub('upsamp','seg',images[i]))
            mytx = ants.registration(fixed = fix,
                                    moving = mov,
                                    type_of_transform = 'Rigid'
                                    )
            warpedmask = ants.apply_transforms(fixed = fix_mask,
                                               moving = mov_mask,
                                               transformlist = mytx['fwdtransforms'],
                                               interpolator = 'nearestNeighbor'
                                              )
            io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',images[i])),warpedmask.numpy())
            io.imsave(re.sub('seg','seg_warped',re.sub('upsamp','seg',re.sub('_0001','',images[i]))),fix_mask.numpy())
            #print(2 * np.sum(warpedmask.numpy()*fix_mask.numpy())/(np.sum(warpedmask.numpy())+np.sum(fix_mask.numpy())))

 66%|████████████████████████████████████████████████████████████████████████████████████▍                                          | 133/200 [01:28<00:44,  1.49it/s]


KeyboardInterrupt: 

## Save Matlab .mat file of registered images

In [13]:
directory_seg = Path('matt_preds')
images = list(directory_seg.glob('*_0001_seg_warped.tif'))
images = sorted([x.as_posix() for x in images])

In [14]:
for image in tqdm(images):
    if not os.path.exists(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image))):
        img_0001 = io.imread(image)
        img = io.imread(re.sub('_0001','',image))
        seg = img*img_0001
        seg = (seg==1)*1
        seg = seg.astype('int8')
        seg = fill_holes(seg)
        savemat(re.sub('_seg_warped.tif','_seg_warped_single.mat',re.sub('_0001','',image)),{'FinalImage':fill_holes(binary_dilation(seg))})
        
        

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 128/128 [00:00<00:00, 2484.66it/s]


# Generate Graphs

In [11]:
directory = Path('matt_preds')
files_seg_0001 = directory.glob('*_0001_seg_warped.tif')
files_seg_0001 = sorted([x.as_posix() for x in files_seg_0001])
len(files_seg_0001)

129

In [16]:
for file_0001 in tqdm(files_seg_0001):
    file = file_0001
    skel_file = re.sub('_0001_seg_warped.tif','_skel_warped_single.mat',file)
    #seg = io.imread(file)
    skel = loadmat(skel_file)['FilteredImage']
    #skel = skeletonize_3d(skel)
    #io.imsave(re.sub('_single.mat','_single.tif',file),skel)
    #dst_tsfm = distance_transform_edt(seg)
    #dst_tsfm[dst_tsfm==0]=0.00001
    #skel_dst = skel*dst_tsfm
#
    #
    #np.save(re.sub('seg','dst_skel',file),skel_dst)
    #io.imsave(re.sub('_seg_warped.tif','_dst_skel_warped.tif',file),skel_dst)
    graph = sknw.build_sknw(skel, multi=False)
    #graph, c0  = skan.csr.skeleton_to_csgraph(skel)
    #print(len(graph.edges))
    
    #print(len(graph_0001.edges))
    #pickle.dump(graph, open(str(re.sub('_seg.npy','.pickle',file)), 'w'))
    #nx.write_pajek(graph,re.sub('_seg.npy','.pajek',file))
    
    #nx.write_gpickle(graph,re.sub('_0001_seg_warped.tif','_warped.pickle',file))
    print(len(graph.edges))

  1%|█                                                                                                                                | 1/129 [00:00<01:13,  1.74it/s]

7


  2%|██                                                                                                                               | 2/129 [00:01<01:21,  1.55it/s]

1168


  2%|███                                                                                                                              | 3/129 [00:01<01:21,  1.55it/s]

...

295


 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████ | 128/129 [01:23<00:00,  1.48it/s]

327


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 129/129 [01:23<00:00,  1.54it/s]

187


In [15]:
len(graph.edges)

7

In [69]:
i=15
nodes_deg_1 = [node for (node, val) in graph.degree() if val == 1]
graph[list(graph.neighbors(nodes_deg_1[i]))[0]][nodes_deg_1[i]]['weight']

243.4288327007428